# Learning of the Transmitter Orientation via Gradient Descent

This notebook reproduces the results for the example application "Optimization of transmitter orientation" in the paper [Sionna RT: Differentiable Ray Tracing for Radio Propagation Modeling](https://arxiv.org).

It requires [Sionna](https://github.com/NVlabs/sionna) v0.16 or later.


## Imports and GPU Configuration

In [ ]:
# Set some environment variables
import os
gpu_num = "" # GPU to be used. Use "" to use the CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppress some TF warnings
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    os.system("pip install sionna")
    import sionna

# Configure GPU
import tensorflow as tf
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_memory_growth(gpus[0], True)
#     except RuntimeError as e:
#         print(e)

# Avoid warnings from TensorFlow
import warnings
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore')

# Other imports
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera

# Fix the seed for reproducible results
tf.random.set_seed(1)

pre_optimisation_cm = []
post_optimisation_cm = []

## Configure the Scene and Generate Reference Data

In this example, we load the scene "etoile" which is already available in Sionna and then place a single transmitter **with trainable orientation** within the scene.


In [ ]:
# Load the scene
#scene = load_scene(sionna.rt.scene.etoile)
# Load integrated scene
scene = load_scene("/Users/andreamaestri/Desktop/tectwin/GEO/geoloc-rf/scenes/accenturePOC/accenturePOC.xml") 
scene.frequency = 3.5e9
# Set the scattering coefficient of the radio material in the scene
# to a non-zero value to enable scattering

In [ ]:
scene.get("elm__4").radio_material = "itu_concrete"

In [ ]:
scene._clear()

In [ ]:
# Configure the transmit array
scene.tx_array = PlanarArray(num_rows=1, num_cols=16,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V",
                             polarization_model=2)

# Configure the receive array (use to compute the coverage map)
scene.rx_array = PlanarArray(num_rows=1, num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V",
                             polarization_model=2)

# Create a transmitter and add it to the scene
#transmitter_coordinates = [[-37.47,-46.16,6],[-30.13,-27.22,6],[-30.2714, -28,6],[54.74, -99.,6], [68.12, -82.21, 6], [13.37, -43.53,6],[37.13, -36.03,6]]
#transmitter_coordinates = [[-8.05184,23.06,6.91],[11.39,16.05,0.9185],[31.43, 30.23, 6.93]]
transmitter_coordinates = [np.array([-4.18, 38.12, 16]), np.array([22.96, 16.6, 16]), np.array([43.58, -1.604, 16])]
#transmitter_coordinates = [np.array([-4.18, 38.12, 16]), np.array([43.58, -1.604, 16])]


tx = Transmitter(f"tx0", position=transmitter_coordinates[0],
                    orientation=tf.Variable([0.0, 0.0, 0.0], tf.float32)) # Trainable orientation

scene.add(tx)
tx = Transmitter(f"tx1", position=transmitter_coordinates[1],
                    orientation=tf.Variable([0.0, 0.0, 0.0], tf.float32)) # Trainable orientation

scene.add(tx)
tx = Transmitter(f"tx2", position=transmitter_coordinates[2],
                    orientation=tf.Variable([0.0, 0.0, 0.0], tf.float32)) # Trainable orientation

scene.add(tx)

# Render the scene
# The transmitter is indicated by a blue ball
# cam = Camera("my_cam", position=(0,0,15))
# scene.add(cam)
# cam.look_at([0,0,0])
# scene.render(cam);

Next, we will compute and show a coverage map for the transmitter `"tx"`. The coverage map corresponds to the average received power into small rectangular regions (or *cells*) of a plane, which, by default, is parallel to the XY plane, and sized such that it covers the entire scene with an elevation of $1.5$ m.

The coverage map is divided into cells of size ``cm_cell_size`` (in meters). The smaller the size of the cells, the more precise the coverage map.

In [ ]:
target_center = np.array([-15,  -15, 1.5]) # Center
target_size = np.array([200,200]) # Size
target_orientation = np.array([0,0,0]) # Orientation: parallel to XY
cm_cell_size = np.array([1,1]) # Each cell is 2mx2m

with tf.GradientTape() as tape:
    # Compute coverage of the target area
    target_cm = scene.coverage_map(cm_center=target_center,
                                   cm_orientation=target_orientation, 
                                   cm_size=target_size, # Target area 
                                   cm_cell_size=cm_cell_size,
                                   diffraction=False, scattering=False, # Enable diffraction and scattering in addition to reflection and LoS
                                   check_scene=False) # Don't check the scene prior to compute to speed things up
    
    rate = tf.reduce_mean(tf.math.log(1. + target_cm.as_tensor()))/tf.math.log(2.)
    loss = -rate
    
# Compute gradients and apply through the optimizer
grads = tape.gradient(loss, tape.watched_variables())

In [ ]:
# Rectangle defining the target area
target_center = np.array([-15,  -15, 1.5]) # Center
target_size = np.array([200,200]) # Size
target_orientation = np.array([0,0,0]) # Orientation: parallel to XY
cm_cell_size = np.array([1,1]) # Each cell is 2mx2m
cm = scene.coverage_map(
                        cm_center=target_center,
                        cm_orientation=target_orientation, 
                        cm_size=target_size,
                        num_samples=10e6, # Reduce if your GPU does not have enough memory
                        cm_cell_size=cm_cell_size,
                        diffraction=False, scattering=False,  # Enables diffraction and scattering in addition to reflection and LoS
                        check_scene=False) # Don't check the scene prior to compute to speed things up)
print(cm.as_tensor().shape)

In [ ]:
#scene.preview(coverage_map=cm, cm_tx=0)

In [ ]:
max1 = 10*np.log10(np.max(cm.as_tensor().numpy()[0]))
indexes = np.where(10*np.log10(cm.as_tensor().numpy()[0])==max1)

10*np.log10(np.max(cm.as_tensor().numpy()[0])/1e-12)

In [ ]:
cm.show(tx=0);
cm.show(tx=1);
cm.show(tx=2);

The shape of the coverage map is ``[num_tx, num_cells_y, num_cells_x]``

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and a set of subplots
fig, axs = plt.subplots(1,3, figsize=(18, 6))

cm_0 = 10*np.log10(np.where(cm[0]==0, 1e-12, cm[0]))
cm_1 = 10*np.log10(np.where(cm[1]==0, 1e-12, cm[1]))
cm_2 = 10*np.log10(np.where(cm[2]==0, 1e-12, cm[2]))

sns.heatmap(cm_0, ax=axs[0], vmin=-150, vmax=-40)
sns.heatmap(cm_1, ax=axs[1], vmin=-150, vmax=-40)
sns.heatmap(cm_2, ax=axs[2], vmin=-150, vmax=-40)

# Adjust layout
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
import numpy as np
cm_0 = np.where(cm[0]==0, 1e-12, cm[0])
cm_1 = np.where(cm[1]==0, 1e-12, cm[1])
cm_2 = np.where(cm[2]==0, 1e-12, cm[2])
N=0
SINR_0 = cm_0 / (cm_1 + cm_2 + N)
SINR_1 = cm_1 / (cm_0 + cm_2 + N)
SINR_2 = cm_2 / (cm_0 + cm_1 + N)

sns.heatmap(10 * tf.math.log(SINR_0))
plt.show()
sns.heatmap(10 * tf.math.log(SINR_1))
plt.show()
sns.heatmap(10 * tf.math.log(SINR_2))
plt.show()

# Define the threshold value
up_threshold = 13

# Create the mask
mask_0 = np.where(SINR_0 > up_threshold, 1, 0)
mask_1 = np.where(SINR_1 > up_threshold, 1, 0)
mask_2 = np.where(SINR_2 > up_threshold, 1, 0)
print(sum(sum(mask_0)) + sum(sum(mask_1)) + sum(sum(mask_2)))
sns.heatmap(mask_0)
plt.show()
sns.heatmap(mask_1)
plt.show()
sns.heatmap(mask_2)
plt.show()

sns.heatmap(mask_0+mask_1+mask_2)
plt.show()

In the above figure, white areas are not hit by any ray. This usually implies that the coverage is negligible in these areas.

## Learn the Orientation of the Transmitter

We will now optimize through gradient descent the orientation of the transmitter to improve the coverage in an area defined by a rectangle with the following properties. Note that these properties are defined in the scene coordinate system.

Let's visualize the target area on the coverage map. We first define a utility function to highlight the target area.

In [ ]:
def visualize_target_area(ax, cell_size, center, size):
    # ax: Pyplot.Axes object on which to show the coverage map
    # cell_size : Size of the cells of the coverage map
    # center : Center of the target area
    # Size : Size of the target area
    def scene2cm(p):
        # Change coordinates p : (x,y) from scene to coverage map system
        shift = scene.center[:2].numpy() - scene.size[:2].numpy()*0.5
        print(p)
        print(shift)
        p = p - shift
        print(p)
        p = np.floor(p/cell_size)
        print(p)
        return p

    # xy is the bottom left corner of the rectangle defining the target area
    xy = scene2cm(center[:2] - size[:2]*0.5)
    size_cm = np.floor(size/cell_size)
    rect = patches.Rectangle(xy, size_cm[0], size_cm[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

Next, we optimize the coverage of the target area through gradient descent with respect to the orientation of the transmitter. To that end, during the optimization process, coverage is only computed for the target area.

In [ ]:
import numpy as np
import tensorflow as tf

target_center = np.array([-15, -15, 1.5])  # Center
target_size = np.array([200, 200])  # Size
target_orientation = np.array([0, 0, 0])  # Orientation: parallel to XY
cm_cell_size = np.array([1, 1])  # Each cell is 1mx1m

# Assuming scene.transmitters['tx0'], scene.transmitters['tx1'], scene.transmitters['tx2'] are the variables to watch
tx0_orientation = scene.transmitters['tx0'].orientation
tx1_orientation = scene.transmitters['tx1'].orientation
tx2_orientation = scene.transmitters['tx2'].orientation

# Configure an SGD optimizer
optimizer = tf.keras.optimizers.legacy.RMSprop(0.1)

# Number of training steps
num_steps = 10

# Weighting factors for the loss terms
weight_sinr = 1
weight_coverage = 0

def train_step():
    """A single training step"""
    with tf.GradientTape() as tape:
        
        # Compute coverage of the target area
        target_cm = scene.coverage_map(cm_center=target_center,
                                       cm_orientation=target_orientation, 
                                       cm_size=target_size,  # Target area 
                                       cm_cell_size=cm_cell_size,
                                       diffraction=True, scattering=True,  # Enable diffraction and scattering in addition to reflection and LoS
                                       check_scene=False)  # Don't check the scene prior to compute to speed things up
        
        # Convert to tf.float32 to ensure dtype consistency
        target_cm_tensor = target_cm.as_tensor()
        
        # Compute cm_0, cm_1, cm_2 using TensorFlow functions
        cm_0 = tf.where(target_cm_tensor[0] == 0, 1e-12, target_cm_tensor[0])
        cm_1 = tf.where(target_cm_tensor[1] == 0, 1e-12, target_cm_tensor[1])
        cm_2 = tf.where(target_cm_tensor[2] == 0, 1e-12, target_cm_tensor[2])
        
        # Assuming noise power N is 1e-12
        N = 1e-12
        
        # Calculate SINR for each transmitter
        SINR_0 = cm_0 / (cm_1 + cm_2 + N)
        SINR_1 = cm_1 / (cm_0 + cm_2 + N)
        SINR_2 = cm_2 / (cm_0 + cm_1 + N)
        
        # Compute SINR loss as the negative log of the SINR, averaged over all cells
        sinr_loss = -tf.reduce_mean(tf.math.log(1+SINR_0 + 1e-12) + tf.math.log(1+SINR_1 + 1e-12) + tf.math.log(1+SINR_2 + 1e-12))
        
        # Compute coverage rate
        scaling = 1e10
        rate = tf.reduce_mean(tf.math.log(1. + target_cm_tensor * scaling)) / tf.math.log(2.0)
        coverage_loss = -rate  # Note the negative sign to make it a loss
        # Combine SINR loss and coverage loss
        loss = weight_sinr * sinr_loss + weight_coverage * coverage_loss
    
    # Compute gradients and apply through the optimizer
    grads = tape.gradient(loss, tape.watched_variables())
    optimizer.apply_gradients(zip(grads, tape.watched_variables()))
    return loss

for step in range(num_steps):
    loss = train_step()
    print(f"Training step {step} - Loss: {loss.numpy():.2E} - tx orientation: {scene.transmitters['tx0'].orientation.numpy(), scene.transmitters['tx1'].orientation.numpy()}", end='\r'),# scene.transmitters['tx2'].orientation.numpy()}", end='\r')


Let's now compute the new coverage map over the entire scene with the optimized orientation.

In [ ]:
cm_new = scene.coverage_map(
                            cm_center=target_center,
                            cm_orientation=target_orientation, 
                            cm_size=target_size, # Target area 
                            cm_cell_size=cm_cell_size,
                            num_samples=10e6, # Reduce if your GPU does not have enough memory
                            diffraction=False, scattering=False, # Enable diffraction and scattering in addition to reflection and LoS
                            check_scene=False) # Don't check the scene prior to compute to speed things up

Finally, the coverage map before and after optimization are shown, with the target area highlighed.

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and a set of subplots
fig, axs = plt.subplots(1,3, figsize=(18, 6))

cm_0 = 10*np.log10(np.where(cm_new[0]==0, 1e-12, cm_new[0]))
cm_1 = 10*np.log10(np.where(cm_new[1]==0, 1e-12, cm_new[1]))
cm_2 = 10*np.log10(np.where(cm_new[2]==0, 1e-12, cm_new[2]))

sns.heatmap(cm_0, ax=axs[0], vmin=-150, vmax=-40)
sns.heatmap(cm_1, ax=axs[1], vmin=-150, vmax=-40)
sns.heatmap(cm_2, ax=axs[2], vmin=-150, vmax=-40)

# Adjust layout
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
# Display the initial coverage map
fig = cm.show(tx=0, vmin=-150, vmax=-40)
fig.suptitle("Before")
# Highlight the target area
#visualize_target_area(fig.axes[0], cm_cell_size, target_center, target_size,)

# Display the optimized coverage map
fig = cm_new.show(tx=0, vmin=-150, vmax=-40)
fig.suptitle("After")
# Highlight the target area
#visualize_target_area(fig.axes[0], cm_cell_size, target_center, target_size)

In [ ]:
np.where(np.array([1,2,3,4,5]) < 3, 1, 0)
200*200

In [ ]:
import numpy as np
cm_0 = np.where(cm_new[0]==0, 1e-12, cm_new[0])
cm_1 = np.where(cm_new[1]==0, 1e-12, cm_new[1])
cm_2 = np.where(cm_new[2]==0, 1e-12, cm_new[2])
N=0
SINR_0 = cm_0 / (cm_1 + cm_2+N)
SINR_1 = cm_1 / (cm_0 + cm_2+N)
SINR_2 = cm_2 / (cm_0 + cm_1 + N)

sns.heatmap(10 * tf.math.log(SINR_0))
plt.show()
sns.heatmap(10 * tf.math.log(SINR_1))
plt.show()
sns.heatmap(10 * tf.math.log(SINR_2))
plt.show()

# Define the threshold value
up_threshold = 13

# Create the mask
mask_0 = np.where(SINR_0 > up_threshold, 1, 0)
mask_1 = np.where(SINR_1 > up_threshold, 1, 0)
mask_2 = np.where(SINR_2 > up_threshold, 1, 0)
print(sum(sum(mask_0)) + sum(sum(mask_1)) + sum(sum(mask_2)))
sns.heatmap(mask_0)
plt.show()
sns.heatmap(mask_1)
plt.show()
sns.heatmap(mask_2)
plt.show()

sns.heatmap(mask_0+mask_1+mask_2)
plt.show()

In [ ]:
scene.preview(coverage_map=cm_new, cm_tx=2)

We can also visualize the coverage maps as an overlay of the scene using the renderer.

In [ ]:
scene.cameras['scene-cam-0'].orientation

In [ ]:
scene.cameras['scene-cam-0'].position

In [ ]:
fig = scene.render('scene-cam-0')
fig.suptitle("Before")

In [ ]:
#Add a camera for viewing the scene from the sky
scene.add(Camera("top-view5", position=(3.452233e+05, 5.685208e+06, 1.830229e+03), orientation=(-41.92037 * np.pi / 180 ,  -0.0509  * np.pi / 180, -90 * np.pi / 180)))
#Rendering with the unoptimized orientation
fig = scene.render("top-view5")
fig.suptitle("Before")

## Discussion

- Other loss functions could be used. For example, if multiple transmitters are considered, maximization of the signal-to-interference ratio could be used as objective function.

- Only the orientation is optimized in this notebook. However, other parameters could be learned as well, such as the geometry of the transmitter/receiver arrays as well as precoding vectors.

- In general, the optimization of a coverage map is a non-convex problem, especially when considering antenna arrays with complex radiation patterns. One could resort to other optimization methods such as Bayesian learning in this case.

In [ ]:
#cmaps = []

In [ ]:
cmaps.append(cm_new)

In [ ]:
settings = [[1.5583165,  0.35283738, 0.10147938], [ 3.0284588,   2.7056296,  -0.13907725],[1.8631023, 2.842613, 0.287598 ]]

In [ ]:
fig = scene.render("top-view2", coverage_map=cmaps[0].as_tensor() + cmaps[1].as_tensor() + cmaps[2].as_tensor(), cm_vmin=-140, cm_vmax=-40)
fig.suptitle("After setting optimisation")

In [ ]:
scene.preview(coverage_map=cmaps[2])

In [ ]:
scene.preview(coverage_map=cmaps[1])